In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
con = sqlite3.connect('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\Dataset\\database.sqlite') 

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3""", con) 

In [3]:
# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 0
    return 1

In [38]:
#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative
print("Number of data points in our data", filtered_data.shape)
filtered_data.head(3)

Number of data points in our data (525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...


# Data Cleaning: Deduplication

In [39]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
print(final.shape)

#Checking to see how much % of data still remains
(final.shape[0] / filtered_data.shape[0])*100

(364173, 10)


69.25890143662969

In [40]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


1    307061
0     57110
Name: Score, dtype: int64

# Text Preprocessing.

1. Removing the html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase
6. Remove Stopwords
7. Finally Snowball Stemming the word (it was obsereved to be better than Porter Stemming)<br>

After which we collect the words used to describe positive and negative reviews

In [7]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [8]:
stop = set(stopwords.words('english'))
sno = nltk.stem.SnowballStemmer('english')

In [9]:
def cleanhtml(sent):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sent)
    return cleantext

In [10]:
def cleanpunc(sent):
    cleaned = re.sub(r'[?|!|\|"|#]', r'', sent)
    cleaned = re.sub(r"[']", r'', cleaned)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned)
    return cleaned

In [11]:
i = 0
str1 = ' '
final_string = []
all_positive_words = [] # store words from positive reviews
all_negative_words = [] # words from negative reviews
s = ''

from tqdm import tqdm

for sent in tqdm(final['Text'].values):
    filtered_sent = []
    sent = cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words) > 2)):
                if(cleaned_words.lower() not in stop):
                    s = (sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sent.append(s)
                    if final['Score'].iloc[i] == 0:
                        all_positive_words.append(s)
                    if final['Score'].iloc[i] == 1:
                        all_negative_words.append(s)
                else:
                    continue
            else:
                continue
                
    str1 = b" ".join(filtered_sent)
    
    final_string.append(str1)
    i = i + 1

100%|█████████████████████████████████████████████████████████████████████████| 364171/364171 [29:21<00:00, 206.71it/s]


In [41]:
final['CleanedText'] = final_string

In [42]:
final.head(3)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...


In [43]:
final['CleanedText'].iloc[0]

b'witti littl book make son laugh loud recit car drive along alway sing refrain hes learn whale india droop love new word book introduc silli classic book will bet son still abl recit memori colleg'

In [131]:
final_shuffled = final.sample(frac = 1).reset_index(drop=True)
X = final_shuffled['CleanedText'].values

y = final_shuffled['Score'].values

# Featurization

## BAG OF WORDS

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(X)
print(count_vect.get_feature_names()[:20])
print("*"*100)

bow = count_vect.transform(X)
bow.shape

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaargh', 'aaaaaaaaagghh', 'aaaaaaah', 'aaaaaaahhhhhh', 'aaaaaaarrrrrggghhh', 'aaaaaah', 'aaaaaahhh', 'aaaaaahhhh', 'aaaaaahhhhh', 'aaaaaahhhhhyaaaaaa', 'aaaaaand', 'aaaaaawwwwwwwwww', 'aaaaah', 'aaaaahhhhhhhhhhhhhhhh']
****************************************************************************************************


(364171, 71624)

In [77]:
from sklearn.model_selection import train_test_split

X_train ,X_test,y_train,y_test = train_test_split(bow, y, test_size = 0.1,stratify = y, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(327753, 71624) (36418, 71624) (327753,) (36418,)


#### Tuning the parameters for best model with a subset dataset

In [78]:
tuning_C = [{'C' : [10**-3,10**-2,10**-1,1]}]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter = 500, verbose = 1, random_state = 1), tuning_C, scoring = 'accuracy',cv = 10, n_jobs = -1)

In [79]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
model.fit(X_train[:10000], y_train[:10000])
print(model.best_estimator_)

elapsed = timeit.default_timer() - start_time
print(elapsed)

6422.7968834


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)
15.634239399999387


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s finished


#### Applying best model to full dataset

In [84]:
best_model = model.best_estimator_

In [86]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
best_model.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time
print(elapsed)

6620.1714734


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


52.30348480000066


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   52.1s finished


In [87]:
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
print('Accuracy ',accuracy_score(y_test, y_pred)*100)

Accuracy  89.35965731231809


In [88]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.85      0.71      5711
           1       0.97      0.90      0.93     30707

    accuracy                           0.89     36418
   macro avg       0.79      0.87      0.82     36418
weighted avg       0.91      0.89      0.90     36418



In [90]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[ 4836,   875],
       [ 3000, 27707]], dtype=int64)

## TF-IDF

In [95]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vect = TfidfVectorizer()
tf_idf_vect.fit(X)
print(tf_idf_vect.get_feature_names()[:20])
print("*"*100)

tf_idf = tf_idf_vect.transform(X)
tf_idf.shape

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaargh', 'aaaaaaaaagghh', 'aaaaaaah', 'aaaaaaahhhhhh', 'aaaaaaarrrrrggghhh', 'aaaaaah', 'aaaaaahhh', 'aaaaaahhhh', 'aaaaaahhhhh', 'aaaaaahhhhhyaaaaaa', 'aaaaaand', 'aaaaaawwwwwwwwww', 'aaaaah', 'aaaaahhhhhhhhhhhhhhhh']
****************************************************************************************************


(364171, 71624)

In [97]:
from sklearn.model_selection import train_test_split

X_train ,X_test,y_train,y_test = train_test_split(tf_idf, y, test_size = 0.1,stratify = y, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(327753, 71624) (36418, 71624) (327753,) (36418,)


#### Tuning the parameters for best model with a subset dataset

In [104]:
tuning_C = [{'C' : [10**-3,10**-2,10**-1,1]}]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter = 500, verbose = 1, random_state = 1),tuning_C,scoring = 'accuracy',cv = 10, n_jobs = -1)


In [106]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
model.fit(X_train[:10000], y_train[:10000])
print(model.best_estimator_)

elapsed = timeit.default_timer() - start_time
print(elapsed)

8373.7224556


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)
8.186264199999641


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished


#### Applying best model to full dataset

In [107]:
best_model = model.best_estimator_

In [109]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
best_model.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time
print(elapsed)

8514.8036388


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


27.11588119999942


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   27.0s finished


In [110]:
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
print('Accuracy ',accuracy_score(y_test, y_pred)*100)

Accuracy  89.01367455653798


In [111]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.88      0.71      5711
           1       0.97      0.89      0.93     30707

    accuracy                           0.89     36418
   macro avg       0.79      0.88      0.82     36418
weighted avg       0.92      0.89      0.90     36418



In [112]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[ 5001,   710],
       [ 3291, 27416]], dtype=int64)

## Word2Vec Average

In [403]:
from sklearn.model_selection import train_test_split

X_train ,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify = y, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(291336,) (72835,) (291336,) (72835,)


In [435]:
# Train your own Word2Vec model using your own text corpus
i = 0
list_of_sentance = []
for sentance in tqdm(X):
    list_of_sentance.append(sentance.split())

100%|███████████████████████████████████████████████████████████████████████| 364171/364171 [00:12<00:00, 29143.81it/s]


In [436]:
# str(b'hello', encoding)
encoding = 'utf-8'
a = str(list_of_sentance[0][0], encoding)

In [437]:
encoding = 'utf-8'
list_to_sent_str = []
for sent in tqdm(list_of_sentance):
    s = []
    for w in sent:
        temp = str(w, encoding)
        s.append(temp)
    list_to_sent_str.append(s)

100%|███████████████████████████████████████████████████████████████████████| 364171/364171 [00:15<00:00, 23308.38it/s]


In [438]:
list_to_sent_str[0][0]

'good'

In [439]:
w2v_model = Word2Vec(list_to_sent_str, min_count=5, size=50, workers=4)
    
print(w2v_model.wv.most_similar('great'))
    
print('='*50)
    
print(w2v_model.wv.most_similar('worst'))

[('good', 0.8602370619773865), ('fantast', 0.8523843288421631), ('terrif', 0.8456414937973022), ('excel', 0.824141263961792), ('awesom', 0.8039730787277222), ('wonder', 0.7729260921478271), ('perfect', 0.7607808113098145), ('nice', 0.7421234846115112), ('fabul', 0.6974979639053345), ('decent', 0.6518517732620239)]
[('nastiest', 0.8452757596969604), ('greatest', 0.7562000751495361), ('disgust', 0.7348315715789795), ('best', 0.7157772779464722), ('horribl', 0.6895779371261597), ('wors', 0.6831773519515991), ('aw', 0.6773359775543213), ('nicest', 0.6574856638908386), ('horrid', 0.6540610790252686), ('terribl', 0.6529087424278259)]


In [440]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words)) # as min_count = 5
print("*"*50)
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  21938
**************************************************
sample words  ['good', 'arriv', 'fast', 'consum', 'speak', 'would', 'recommend', 'everyon', 'got', 'sucker', 'today', 'hooray', 'cours', 'almost', 'love', 'yummi', 'earth', 'lolli', 'wont', 'bother', 'review', 'flavor', 'fantast', 'sweet', 'pleasant', 'fruiti', 'definit', 'worth', 'wait', 'previous', 'purchas', 'uneven', 'assort', 'like', 'mani', 'other', 'mention', 'wed', 'get', 'abund', 'sour', 'appl', 'grape', 'blueberri', 'mango', 'tango', 'never', 'order', 'counter', 'contain']


In [441]:
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in tqdm(list_to_sent_str): # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length 50, 
                            # you might need to change this to 300 if you use google's w2v
    cnt_words = 0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

100%|█████████████████████████████████████████████████████████████████████████| 364171/364171 [16:12<00:00, 374.60it/s]

364171
50


In [443]:
sent_vectors = np.asarray(sent_vectors)
print(sent_vectors.shape)

sent_vectors_df = pd.DataFrame(sent_vectors)

sent_vectors_df.to_csv('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_avg.csv')

(364171, 50)


In [444]:
w2v_avg = pd.read_csv('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_avg.csv')
w2v_avg.drop(w2v_avg.columns[[0]], axis = 1, inplace = True)
w2v_avg.shape

(364171, 50)

### Splitting DATA

In [445]:
from sklearn.model_selection import train_test_split

X_train ,X_test, y_train, y_test = train_test_split(w2v_avg, y, test_size = 0.1,stratify = y, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(327753, 50) (36418, 50) (327753,) (36418,)


#### Tuning the parameters for best model with a subset dataset

In [446]:
tuning_C = [{'C' : [10**-3,10**-2,10**-1,1, 10,100]}]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter = 500, verbose = 1, random_state = 1), tuning_C, scoring = 'accuracy',cv = 10, n_jobs = -1)

In [447]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
model.fit(X_train[:50000], y_train[:50000])
print(model.best_estimator_)

elapsed = timeit.default_timer() - start_time
print(elapsed)

31217.6321829


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LogisticRegression(C=1, class_weight='balanced', dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)
15.354153599997517


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s finished


#### Applying best model to full dataset

In [448]:
best_model = model.best_estimator_

In [449]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
best_model.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time
print(elapsed)

31235.9691926


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


5.413583099998505


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.1s finished


In [450]:
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
print('Accuracy ',accuracy_score(y_test, y_pred)*100)

Accuracy  83.01664012301609


In [451]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.48      0.85      0.61      5711
           1       0.97      0.83      0.89     30707

    accuracy                           0.83     36418
   macro avg       0.72      0.84      0.75     36418
weighted avg       0.89      0.83      0.85     36418



In [452]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[ 4840,   871],
       [ 5314, 25393]], dtype=int64)

### TF-IDF W2V

In [453]:
model = TfidfVectorizer()
model.fit(X)
# we are converting a dictionary with word as a key, and the idf as a value
dictionary = dict(zip(model.get_feature_names(), list(model.idf_)))

In [454]:
# TF-IDF weighted Word2Vec
tfidf_feat = model.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

In [455]:
len(tfidf_feat), len(w2v_words)

(71624, 21938)

In [456]:
tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in tqdm(list_to_sent_str): # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words and word in tfidf_feat:
            vec = w2v_model.wv[word]
            
            tf_idf = dictionary[word]*(sent.count(word)/len(sent)) # this line evaluates idf * tf
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

100%|████████████████████████████████████████████████████████████████████████| 364171/364171 [6:17:14<00:00, 16.09it/s]


In [457]:
tfidf_sent_vectors = np.asarray(tfidf_sent_vectors)
tfidf_sent_vectors_df = pd.DataFrame(tfidf_sent_vectors)

tfidf_sent_vectors_df.to_csv('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_tfidf.csv')

In [458]:
w2v_tfidf = pd.read_csv('E:\\IIT Kanpur\\Placements\\Projects\\Amazon Fine Food Reviews\\w2v_tfidf.csv')
w2v_tfidf.drop(w2v_tfidf.columns[[0]], axis = 1, inplace = True)
w2v_tfidf.shape

(364171, 50)

### Splitting DATA

In [469]:
from sklearn.model_selection import train_test_split

X_train ,X_test, y_train, y_test = train_test_split(w2v_tfidf, y, test_size = 0.3,stratify = y, random_state = 1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(254919, 50) (109252, 50) (254919,) (109252,)


#### Tuning the parameters for best model with a subset dataset

In [470]:
tuning_C = [{'C' : [10**-3,10**-2,10**-1,1, 10,100]}]

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

model  = GridSearchCV(LogisticRegression(class_weight = 'balanced', max_iter = 500, verbose = 1, random_state = 1), tuning_C, scoring = 'accuracy',cv = 10, n_jobs = -1)

In [471]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
model.fit(X_train[:50000], y_train[:50000])
print(model.best_estimator_)

elapsed = timeit.default_timer() - start_time
print(elapsed)

114505.4206121


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=500, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=1,
                   warm_start=False)
15.427250199994887


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished


#### Applying best model to full dataset

In [472]:
best_model = model.best_estimator_

In [473]:
import timeit
start_time = timeit.default_timer()
print(start_time)

# code you want to evaluate
best_model.fit(X_train, y_train)

elapsed = timeit.default_timer() - start_time
print(elapsed)

114526.560591


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


3.3945290999981808


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s finished


In [474]:
from sklearn.metrics import accuracy_score

y_pred = best_model.predict(X_test)
print('Accuracy ',accuracy_score(y_test, y_pred)*100)

Accuracy  80.82964156262585


In [475]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.44      0.82      0.57     17133
           1       0.96      0.81      0.88     92119

    accuracy                           0.81    109252
   macro avg       0.70      0.81      0.72    109252
weighted avg       0.88      0.81      0.83    109252



In [468]:
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_pred)
conf_mat

array([[ 4673,  1038],
       [ 6004, 24703]], dtype=int64)